# QKV-Attention and Masking

QKV-Attention (scaled-dot-product and multi-headed attention), Masking and Positional Encoding are the main 3 concepts that in my opinion made transformers different from their precursor recurrent neural networks in 2017

## Scaled Dot-Product Attention 

Scaled Dot-Product Attention is used inside Multi-Head Self-Attention.

In the original [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) paper it is expressed as:

$$
\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{softmax}(\frac{\mathbf{Q}\mathbf{K}^\top}{\sqrt{n}})\mathbf{V}
$$

Q stands for query, K for key and V for value vectors respectively. The expression inside the softmax is called the scores. You can think of Q, K and V as serving 3 distinct roles for the same sentence, sequence encoding, or sequence of vectors. Their shapes are:

1. q (q_seq_len, emb_dim)
2. k (kv_seq_len, emb_dim)
3. v (kv_seq_len, emb_dim)


K vectors are like V vector's representatives, interacting with vectors in Q on V's behalf. The larger the dot product between K_i and Q_j, the greater the contribution V_i will make to the new V_j. 

Q does not have to come from the same sequnce encoding as K and V. For example in encoder decoder transformers their is a layer sometimes called *cross attention* where Q is from one sequence and K, V is from another. The Q sequence does not need to be the same sequence length of the K-V sequence, but K and V do have to come from the same sequence. When Q, K, V come from the same sequence, this is called self-attention. The inputs to the multiheadattn function below take (batch_size, sequence_len, emb_dim) shape tensors that represent sequences in the order of the tensors that are transformed into Q, K, and V in that order.


Decoder Encoder Attention
`en_attn,en_scores=multiheadattn(decode,encode,encode,encodemask)`

Self Attention
`self_attn,self_scores=multiheadattn(decode,decode,decode,decodemask)` 

The image below depicts from left right: 

1. a Q tensor of shape (Q sequence_length, embedding_dimension = (2,3)

2. a transposed K tensor of shape (embedding_dimension, KV sequence_length) = (3,2)

3. a score tensor of shape (Q sequence_length, KV sequence_length)

<img src="../saved/images/scorematrix.png">

In the above image, the rightmost tensor is the scores tensor. The resulting scores tensor's vertical axis (dim=0) corresponds to the sequence length of Q and the horizontal axis (dim=1) corresponds to the sequence length of K. The above image represented in code is:

```python
k_T = k.transpose(-2, -1) 

scores = torch.matmul(q, k_T)
```
PyTorch's matmul operation, performs matrix multiplication on the last two dimensions of our tensors. torch.bmm does something similar.

What is not shown is that between the top image and the bottom image, the scores tensor is scaled by dividing by the square root of the embedding dimension and a softmax function normalizes each element of the Q sequence to be a probability distribution over each element of the K-V sequence. The Horizontal axis is the last dimension, thus dim = -1 is passed to `F.softmax`. 

```python
scores = scores / math.sqrt(emb_dim) 

softscores = F.softmax(scores, dim=-1)
```

The image below depicts from left to right:

1. the scaled and softmax normalized `softscores` on the left.

2. a V tensor of shape (V sequence_length, embedding_dimension = (2,3)

3. a re-composed Q tensor, recomposed from portions of V. This output tensor has shape (Q sequence_length, emb_dim)

<img src="../saved/images/contextmatrix.png">

One way to interpret the first row of the score matrix is that it is saying "rebalance the word 'Thinking' by taking 80% of the magnitude of 'Thinking' and add it to 20% of the value of 'Machines'""

The length of Q and K-V above happen to both be 2, (Q sequence_length, K sequence_length) = (2,2), but remember that the sequence length of Q and K-V can be different. In the line  `output = torch.matmul(scores, v)` we are essentilly performing the operations of shape <font color='red'>score matrix (q_seq_len, kv_seq_len)</font> X <font color='lightblue'>values (kv_seq_len, dim)</font> = <font color='blue'>context values (q_seq_len, dim)</font> for every sample in the batch and for every head. The final shape and sequence length of Scaled-Dot-Product Attention on Q, K V is (Q sequence_length, emb_dim) since the image represents an operation of shape: 

(Q sequence_length, KV sequence_length) x (KV sequence_length, emb_dim) =
(Q sequence_length, emb_dim)

The demo below uses a toy input, simulating a batch with samples, sequence length, and a embedding dimension. 

The entire equation is carried out in the pytorch tensor example below

```python
scores
tensor([[[[ 0.2509,  0.0725,  0.0990],
          [ 0.0483, -1.8634, -1.4245]]]])
softscores
tensor([[[[0.3710, 0.3104, 0.3187],
          [0.7262, 0.1073, 0.1665]]]])
output
tensor([[[[ 0.5732,  0.4398,  0.0379,  0.4533],
          [ 1.0041,  0.5920, -0.1833,  0.6731]]]])
```

notice that the values in softscores sum to 1.0 across
the horizontal axis, this axis is `dim=-1` in 
`softscores = F.softmax(scores, dim=-1)` 
if you do `softscores = F.softmax(scores, dim=-2)` 

you will mistakenly get instead

```python
softscores
tensor([[[[0.5505, 0.8739, 0.8211],
          [0.4495, 0.1261, 0.1789]]]])
```

On the Y-axis are the number of words in the Q sequence. Each row is a word in Q. So the length of this dimension is the same as the number of words in Q. 

On the X-axis is how much each Q-word relies on each K-V word. So the length of this dimension is the same as the number of words in K or V. 
For example if the index (1,2) position is 0.7 it means that the 2nd word in Q replies 70% on the 3rd word in K-V. 

In [6]:
import math

import numpy as np

import torch
import torch.nn.functional as F

torch.manual_seed(42)

batch_size = 1
num_heads = 1
q_seq_len = 2
k_seq_len = 3
v_seq_len = 3
emb_dim = 4

q = torch.randn(batch_size, num_heads, q_seq_len, emb_dim)

k = torch.randn(batch_size, num_heads, k_seq_len, emb_dim)
v = torch.randn(batch_size, num_heads, v_seq_len, emb_dim)

k_T = k.transpose(-2, -1) 

scores = torch.matmul(q, k_T) / math.sqrt(emb_dim) 

print('scores')
print(scores)

softscores = F.softmax(scores, dim=-1)

print('softscores')
print(softscores)

output = torch.matmul(softscores, v)

print('output')
print(output)

scores
tensor([[[[ 0.2509,  0.0725,  0.0990],
          [ 0.0483, -1.8634, -1.4245]]]])
softscores
tensor([[[[0.3710, 0.3104, 0.3187],
          [0.7262, 0.1073, 0.1665]]]])
output
tensor([[[[ 0.5732,  0.4398,  0.0379,  0.4533],
          [ 1.0041,  0.5920, -0.1833,  0.6731]]]])


In [1]:
def dot_product_attention(
    q, k, v, dim, 
    mask=None, dropout=None, explain=False
    ):

    ''' Function that performs scaled dot product attention on
    tensors Q, K and V. each of shape:
    
    (batch_size, num_heads, sequence_length, dim)
    
    where k and v are expected to have the same sequence_length
    but q and k do not need to have the same sequence length.
    
    matrix multiplication is done using the last two dimensions of 
    the input tensors:
    
    (batch_size, num_heads, q_seq_len, dim) 
    X 
    (batch_size, num_heads, dim, k_seq_len)
    = 
    (batch_size, num_heads, q_seq_len, k_seq_len)
    '''

    k = k.transpose(-2, -1) 

    if explain: print('q, k', q.shape, k.shape)

    scores = torch.matmul(q, k) / math.sqrt(dim)  

    if explain: print('scores.shape', scores.shape)

    if mask is not None:

        mask = mask.unsqueeze(1)
        if explain: print('mask.shape', mask.shape)
        scores = scores.masked_fill(mask == 0, -1e9) 

    softscores = F.softmax(scores, dim=-1)

    if dropout is not None: softscores = dropout(softscores)

    #(batch_size,num_heads,q_seq_len, k_seq_len)
    #X
    #(batch_size,num_heads,v_seq_len,dim_v)
    
    output = torch.matmul(softscores, v)

    # output (batch_size, num_heads, q_seq_len, dim_v)
    return output, scores 

## Multi-Head Self-Attention

The `MultiHeadAttention()` module takes in a sequence of vectors and outputs a sequence of vectors of the same shape. What it does to the sequence is modify each vector in such a way that it takes into account, to different degrees, the other vectors in the sequence. 

Consider the all the different meanings of the word "date", spelled the same way

- Her favorite fruit is a date.
- Carson took Vicki out on a date.
- Not to date myself, but I remember listening to cassette tapes.
- 10/26/19, thats the date of our wedding

The only way for you to know which of these meanings the word "date" is being used as, is by using the the other words in the sentence, ie the context. 

For every word in the sentence, the authors of the Transformer came up with the "attention head". Here is one example from the paper [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) of how one head is using the other words to modify the representation of the word "it's"

<img src="../saved/images/onehead.png">

Several heads are used, to allow different types of attention, the image below shows 3 types of attention for different relationships between words "who", " Did what?" and "To whom?"

<img src="../saved/images/multihead.png">

Each attention head works the same way, by transforming each vector in the sequence into another vector space. Those vectors are q, k and v, meaning the query, key and value vectors. Where the dot product between the "it's" query vector and the "law" key vector is proportional to the strength of the attention between "it's" and "law"

[Jay Alammar](http://jalammar.github.io/illustrated-transformer/) does an excellent job in illustrating this in in his blog:

Alammar, Jay (2018). The Illustrated Transformer [Blog post]. Retrieved from https://jalammar.github.io/illustrated-transformer/

He uses the example source sequence <font color='green'>"Thinking Machines"</font> represented by the 2x4 matrix <font color='green'>X</font>. 2x4 because there are 2 words, each word represented by a 4 dimensional vector. The q, k and v vectors are each 3 dimensional. 

Going from x to q occurs by doing a matrix multiplication. If X is shape (2x4), you can use a matrix W of shape (4x3) to transform X to a matrix <font color='purple'>Q</font> of shape (2x3) Each row q is for one of the 2 words. Instead of running a for loop through all the heads, it is better to "vectorize" your operations for computational efficiency, by concatenating all your W's together. In the example below we have 3 heads, so instead of doing 3 seprate matrix multiplications with a transofrmation matrix (4x3) we use one large matrix shape (4x9) and slice our heads out of the longer matrix later

<img src="../saved/images/qkvmatrix.png">

in the code below, the operation in the image is represented by the code 

`q = self.q_linear(q)`

`k = self.k_linear(k)`

`v = self.v_linear(v)`

In the forward method of MultiHeadAttention above, the `view()` and `transpose()` operations serve to [re-arrange](https://discuss.pytorch.org/t/tensor-view-is-misleading/20553) the matrices q, k, into the shape **(batch_size, num_heads ,seq_len, dim_qk)**

`concat=attn.transpose(1,2).contiguous().view(batch_size,-1,self.dim_k*self.num_heads)` reshapes the tensor by concatenating all the values from each head to make a tensor shape (batch size, sequence length, dim_v*num_heads)

`output = self.out(concat)` uses a matrix multiplication to tranform each dim_v word vector to one of the same size as the embedding dimensions. The result is that the shape of the attention output is the same as the input. 
(batch size, sequence length, embedding dimensions)

In [4]:
import torch

import torch.nn as nn

class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, emb_dim, dim_k = None, dropout = 0.1):
        
        super().__init__()
        
        self.emb_dim = emb_dim
        self.dim_k = dim_k if dim_k else emb_dim // num_heads
        self.num_heads = num_heads
        self.q_linear = nn.Linear(emb_dim,self.dim_k*num_heads)
        self.k_linear = nn.Linear(emb_dim,self.dim_k*num_heads)
        self.v_linear = nn.Linear(emb_dim,self.dim_k*num_heads)

        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(self.dim_k*num_heads,emb_dim)
    
    def forward(self, q, k, v, mask=None, explain=False):
        '''
        inputs:
            q has shape (batch size, q_sequence length, embedding dimensions)
            k,v are shape (batch size, kv_sequence length, embedding dimensions)
            source_mask of shape (batch size, 1, kv_sequence length)
            
        outputs: sequence of vectors, re-represented using attention
            shape (batch size, q_sequence length, embedding dimensions)
        use:
            The encoder layer places the same source vector sequence into q, k, v 
            and source_mask into mask.
            
            The decoder layer uses this twice, once with decoder inputs as q, k, v 
            and target mask as mask. then with decoder inputs as q, encoder outputs
            as k, v and source mask as mask
        '''
        # k,q,v are each shape (batch size, sequence length, dim_k * num_heads)
        
        batch_size = q.size(0)
        
        q = self.q_linear(q)
        k = self.k_linear(k)
        v = self.v_linear(v)
        
        if explain: print("(batch size, sequence length, dim_k * num_heads)", k.shape)
            
        # k,q,v are each shape (batch size, sequence length, num_heads, dim_k)
        
        k = k.view(batch_size, -1, self.num_heads, self.dim_k)
        q = q.view(batch_size, -1, self.num_heads, self.dim_k)
        v = v.view(batch_size, -1, self.num_heads, self.dim_k)
        
        # transpose to shape (batch_size, num_heads, sequence length, dim_k)
        
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        
        if explain: print("(batch_size,num_heads,qk_seq_length,dim_k)", k.shape)
        if explain: print("(batch_size,num_heads,v_seq_length,dim_k)", v.shape)
            
        # calculate attention using function we will define next
        
        attn, scores = dot_product_attention(q, k, v, self.dim_k, mask, self.dropout, explain)
        
        if explain: print("attn(batch_size,num_heads,seq_length,dim_k)", attn.shape)
            
        # concatenate heads, dim_k = dim_v 
        
        concat = attn.transpose(1,2).contiguous().view(batch_size,-1,self.dim_k*self.num_heads)
        
        if explain: print("concat.shape", concat.shape)
            
        # put through final linear layer
        
        output = self.out(concat)
        
        if explain: print("MultiHeadAttention output.shape", output.shape)
            
        return output, scores

In [8]:
input_vectors = np.asarray([ 
[[0.0, 0.0, 0.1, 0.2, 0.3],[0.0, 0.1, 0.2, 0.3, 0.4],[0.0, 0.2, 0.3, 0.4, 0.5]], 
[[0.0, 0.1, 0.2, 0.3, 0.4],[0.0, 0.2, 0.3, 0.4, 0.5],[0.0, 0.3, 0.4, 0.5, 0.6]]  
])

input_mask = torch.from_numpy(np.ones((2,1,3)) == 1)
input_mask[0,0,-1] = False
input_vectors = torch.from_numpy(input_vectors).float()

print('input_vectors shape\n', input_vectors.shape)
print('---------------------------------------')
print('input_mask\n', input_mask, input_mask.shape)

input_vectors shape
 torch.Size([2, 3, 5])
---------------------------------------
input_mask
 tensor([[[ True,  True, False]],

        [[ True,  True,  True]]]) torch.Size([2, 1, 3])


In [9]:
multiattention = MultiHeadAttention(num_heads=6,emb_dim=5,dim_k=4,dropout=0.0)
output, scores = multiattention(input_vectors,input_vectors,input_vectors,input_mask,explain=True)

(batch size, sequence length, dim_k * num_heads) torch.Size([2, 3, 24])
(batch_size,num_heads,qk_seq_length,dim_k) torch.Size([2, 6, 3, 4])
(batch_size,num_heads,v_seq_length,dim_k) torch.Size([2, 6, 3, 4])
q, k torch.Size([2, 6, 3, 4]) torch.Size([2, 6, 4, 3])
scores.shape torch.Size([2, 6, 3, 3])
mask.shape torch.Size([2, 1, 1, 3])
attn(batch_size,num_heads,seq_length,dim_k) torch.Size([2, 6, 3, 4])
concat.shape torch.Size([2, 3, 24])
MultiHeadAttention output.shape torch.Size([2, 3, 5])


## Masks and Softmax
The last thing to cover in the `MultiHeadAttention()` function is the way the mask is applied to the attention scores and the way the softmax is applied to the attention scores. 

In intuitive words, imagine that the first sample is `thinking machines <pad>`. We dont need to incorporate the token `<pad>` into the representation of "thinking" or "machines" so our mask for this sequence is `[ True,  True, False]`.
In the case of the decoder output, the mask depends on what step in the output we are in. Each output can only attend to the previous outputs so the mask will look something like `[ True]` for the first step and `[ True,  True, True, False]` for the 3rd step etc. This is more relevant for training but not for inference. 

The mask is reshaped into (batch size, 1, 1, sequence length) in the case of the source mask, and (batch size, 1, sequence length, sequence length) in the case of the target mask.

This is [broadcasted](https://medium.com/ai%C2%B3-theory-practice-business/understanding-broadcasting-in-pytorch-ca9e9533f05f) across the scores which are of shape (batch size, number of heads, sequence length, sequence length). Broadcasting will apply the mask across each row of the (sequence length x sequence length) matrix. Think of each row as the being assigned to each token. The first row is for the token "thinking". If this row looks something like `[2.0, 0.1, -100]`, it means that the word "thinking" should be defined mostly by it's own token `2.0`, alittle by the word "machines" `0.1`, and practically not at all by the padding token `-100`. The code `scores = scores.masked_fill(mask == 0, -1e9)` takes the very negative value, `-1e9` and replaces the score values at equivalent position where the mask has `False` or `0` as it's element, with this very negative value `-1.0000e+09`. 

`softscores = F.softmax(scores, dim=-1)` will then rebalance each row such that they all sum to 1.0, `dim=-1` means the last dimension. Think of a matrix that has 2 rows and 3 columns, it is a 2 x 3 matrix. If you softmax across the last dimension of size 3, you are making all 3 of those sum to 1.0 

In the cell below we demonstrate what we have discussed by running the attention function without the mask, getting the unmasked score and applying the mask to the score step by step for demonstration

In [10]:
output, scores = multiattention(input_vectors,input_vectors,input_vectors,explain=False)
print(input_mask, input_mask.shape, scores.shape)
print(scores[0,0,:,:])
print(scores[1,0,:,:])
print("------------------------------------------------------")
mask = input_mask.unsqueeze(1)
print(mask, mask.shape, scores.shape)
scores = scores.masked_fill(mask == 0, -1e9) 
print(scores[0,0,:,:])
print(scores[1,0,:,:])
print("------------------------------------------------------")
softscores = F.softmax(scores, dim=-1)
print(softscores[0,0,:,:])
print(softscores[1,0,:,:])

tensor([[[ True,  True, False]],

        [[ True,  True,  True]]]) torch.Size([2, 1, 3]) torch.Size([2, 6, 3, 3])
tensor([[-0.0193, -0.0198, -0.0204],
        [-0.0266, -0.0269, -0.0272],
        [-0.0339, -0.0339, -0.0339]], grad_fn=<SliceBackward0>)
tensor([[-0.0269, -0.0272, -0.0274],
        [-0.0339, -0.0339, -0.0338],
        [-0.0409, -0.0406, -0.0402]], grad_fn=<SliceBackward0>)
------------------------------------------------------
tensor([[[[ True,  True, False]]],


        [[[ True,  True,  True]]]]) torch.Size([2, 1, 1, 3]) torch.Size([2, 6, 3, 3])
tensor([[-1.9260e-02, -1.9849e-02, -1.0000e+09],
        [-2.6583e-02, -2.6866e-02, -1.0000e+09],
        [-3.3905e-02, -3.3883e-02, -1.0000e+09]], grad_fn=<SliceBackward0>)
tensor([[-0.0269, -0.0272, -0.0274],
        [-0.0339, -0.0339, -0.0338],
        [-0.0409, -0.0406, -0.0402]], grad_fn=<SliceBackward0>)
------------------------------------------------------
tensor([[0.5001, 0.4999, 0.0000],
        [0.5001, 0.4999, 0.000